In [11]:
from selenium import webdriver
from datetime import datetime
from time import sleep
import configparser
import os
import sys
import database_functions as dbf
#import re
#import pandas as pd
import locale
#driver_path_chrome = r'C:\Users\jonat\chromedriver.exe'
#driver_path_edge = r'C:\Users\jonat\msedgedriver.exe'
locale.setlocale(locale.LC_ALL, "ko_KR.UTF-8")


'ko_KR.UTF-8'

In [12]:
config_file_name = 'price_monitoring.config'

#IMPORT CONFIG FROM FILE .CONFIG
#cwd = os.path.dirname(sys.argv[0]) #working on linux server or windows python script
cwd = os.getcwd() #working on windows during python notebook execution
cfg = configparser.ConfigParser()
ini_config_path = os.path.join(cwd,config_file_name)
cfg.read(ini_config_path)
driver_path_chrome = cfg['general_config']['driver_path_chrome']
driver_path_edge = cfg['general_config']['driver_path_edge']
database = cfg['database_config']['database']


In [13]:
def get_price():
    xpath_price = '/html/body/div[2]/section/div[1]/div/div[3]/div[5]/div[1]/div/div[2]/span[1]/strong'
    price_content = driver.find_element_by_xpath(xpath_price).text
    try:
        price_content = price_content.replace('원', '')
    except:
        price_content = price_content
    price = locale.atof(price_content)
    return price

In [14]:
def get_image():
    xpath_image = '/html/body/div[2]/section/div[1]/div/div[1]/div[1]/img'
    image_content = driver.find_element_by_xpath(xpath_image).get_attribute('src')
    return image_content

In [15]:
#connect to database and create table if it doesn't exist
conn, cur = dbf.connect_to_database(database)
columns = "price REAL, time_utc_now TEXT, image_link TEXT, link TEXT"
table_name = "prices"

dbf.create_table(conn, cur, table_name, columns)

Table prices created


In [16]:
url_list = [
    r'https://www.coupang.com/vp/products/6366671555?isAddedCart=',
    r'https://www.coupang.com/vp/products/5056493492?isAddedCart=',
    r'https://www.coupang.com/vp/products/5835511110?isAddedCart=',
    r'https://www.coupang.com/vp/products/6060667011?isAddedCart=',
    r'https://www.coupang.com/vp/products/2530237?isAddedCart=',
    r'https://www.coupang.com/vp/products/266030794?isAddedCart=',
    r'https://www.coupang.com/vp/products/4978269600?isAddedCart='
]
price_list = []
for url in url_list:
    driver = webdriver.Edge(driver_path_edge)

    try:
        driver.get(url)
        sleep(0.5)
        price = get_price()
        time_utc_now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
        image_link = get_image()
        link = url

        data = str(price) + ",'" + time_utc_now + "','" + image_link + "','" + link + "'"

        act_dict = {
            'price': price,
            'time_utc_now': time_utc_now,
            'image_link': image_link,
            'link': link
            }
        price_list.append(act_dict)
        
        dbf.insert_data(conn, cur, table_name, data)
    except:
        pass

    driver.quit()
    sleep(1)
price_list

Data inserted
Data inserted
Data inserted
Data inserted
Data inserted
Data inserted
Data inserted


[{'price': 69900.0,
  'time_utc_now': '20220803074703',
  'image_link': 'https://thumbnail7.coupangcdn.com/thumbnails/remote/492x492ex/image/vendor_inventory/d6e2/022a1986626056c183a71149d2306a170c7f6b98e56ea7c791a1b4b56405.jpg',
  'link': 'https://www.coupang.com/vp/products/6366671555?isAddedCart='},
 {'price': 287780.0,
  'time_utc_now': '20220803074711',
  'image_link': 'https://thumbnail7.coupangcdn.com/thumbnails/remote/492x492ex/image/vendor_inventory/ed86/4001ed4983e31ddf4cd47b6d4474f704176d705acb4d38396112f6927b8e.jpg',
  'link': 'https://www.coupang.com/vp/products/5056493492?isAddedCart='},
 {'price': 489420.0,
  'time_utc_now': '20220803074718',
  'image_link': 'https://thumbnail9.coupangcdn.com/thumbnails/remote/492x492ex/image/vendor_inventory/26af/da9983d79feb7ccbe74407afbc4a2321408c0127a70acf93f2fea20e00bc.jpg',
  'link': 'https://www.coupang.com/vp/products/5835511110?isAddedCart='},
 {'price': 946990.0,
  'time_utc_now': '20220803074726',
  'image_link': 'https://thum

In [17]:

dbf.close_connection(conn)

Connection closed


In [18]:
type(price_list[0]['price'])

float

In [19]:
#driver = webdriver.Edge(driver_path_edge)
#driver.get(r'https://www.coupang.com/vp/products/6366671555?isAddedCart=')
#xpath_image = "/html/body/div[2]/section/div[1]/div/div[1]/div[1]/img"
#image_content = driver.find_element_by_xpath(xpath_image).get_attribute('src')
#image_content

In [20]:
datetime.utcnow().strftime("%Y%m%d%H%M%S")

'20220803074753'